## Toy model fitting
Attempted to perform a dummy regression task on the prepared data to test the pipeline.

In [4]:
import sys
from os import listdir
from os.path import isfile, join
import time
import datetime as dt
from datetime import datetime
import numpy as np
import pandas as pd
import scipy.io
import iisignature
import matplotlib.pyplot as plt

# set flag for file locations
local = True

if local:
    dir_flacs = 'D:/flep/ukdale'
    fnstem_appliance_data = "D:/flep/ukdale/house_1/channel_"

    # for running in gedit
    # sys.path.append('/home/paul/Desktop/cache/dale/code')  
else:
    # dir_flacs = '/scratch/dale_data/UK-DALE-2017/UK-DALE-2017-16kHz/house_1/2016/wk01'
    fnstem_appliance_data = "/scratch/moorep/dale/house_1/channel_"

import read_flac_file as rff



# returns sorted file list from dir_flacs - currently not used
def get_flac_files(dir_flacs):

    onlyfiles = [f for f in listdir(dir_flacs) if isfile(join(dir_flacs, f))]
    return(sorted(onlyfiles))
      


# gets appliance data and returns as a data frame
def get_appliance_data(channel, date_tuple, start_hour, number_of_hours=2):

    if channel == -1:
        fn_dat = "/scratch/moorep/dale/house_1/mains.dat"
    else:
        fn_dat = fnstem_appliance_data + str(channel) + ".dat"

    # set time interval
    ts_start = time.mktime(dt.date(date_tuple[0],date_tuple[1],date_tuple[2]).timetuple()) + 3600*start_hour
    ts_end = ts_start + number_of_hours*3600 

    # read appliance data
    df = pd.read_csv(fn_dat,' ') 
    df.columns = ['ts','watts']
    df = df.loc[(df['ts']>=ts_start) & (df['ts']<ts_end),:]
    
    return(df)




# finds signature for each cycle in the input voltage
def cycle_sigs(voltage, current, deg, logsig=True):
    
    # number of channels (dimensions) in path - time, voltage, current
    m = 2

    # find the signature length using an arbitrary path
    sample_path = np.ones([10,m])
    if logsig:
        s = iisignature.prepare(m,deg)
        len_sig = len(iisignature.logsig(sample_path,s))
    else:
        len_sig = len(iisignature.sig(sample_path,deg))

    # find zero up-crossing indexes for voltage
    zci = np.where(np.diff(np.sign(voltage))>0)[0]
    
#debug
    #print('Limiting zci')
    #zci = zci[0:2]

    sigs = np.empty([len(zci)-1,len_sig]);

    # create a path for each cycle 
    for k,z in enumerate(zci[0:-1]):

        ch1 = voltage[zci[k]:zci[k+1]]
        ch2 = np.cumsum(ch1*(current[zci[k]:zci[k+1]])) 
        #ch2 = current[zci[k]:zci[k+1]]
        #integral = sum(ch1*ch2)
        
        ch0 = np.linspace(0,1,ch1.shape[0]) # time dimension

        path = np.column_stack([ch0, ch2])

        
        # create the signature using iisignature
        if logsig:
            sigs[k,:] = iisignature.logsig(path,s)
        else:    
            sigs[k,:]  = iisignature.sig(path,deg)
        
        #print(ch1[:20],ch2[:20],sigs)
            

    # drop the final zci because it is at the cycle end
    zci = zci[0:-1]
    return(zci, sigs)


# creates the training set given appliance dataframe, cycle signatures and their timestamps
def create_labelled_set(df_appliance, sigs, ts_sigs):
        
    # margin in seconds from the appliance's switch on and off times
    margin = 10

    # preallocate array and truncate later
    labelled_set = np.zeros((sigs.shape[0],sigs.shape[1]+1));

    # create the on/off labels by detecting when the fridge uses more than 20 watts
    df_appliance['on'] = df_appliance['watts'] < 20

    # find the indexes for when the appliance turns on and off
    on_off_indexes = np.where(np.diff(df_appliance['on']))[0]
    print("State switches %d times."%len(on_off_indexes))
    
    c = 0
    
    # set the first label - subsequent labels are obtained by toggling
    label = int(df_appliance['on'].iloc[0])
    
    sig_len = sigs.shape[1]
    ts_appliance = df_appliance['ts']

    # iterate over the appliance on-off periods
    for idx, ooi in np.ndenumerate(on_off_indexes[0:-1]):

        # find the on or off time and select a clean on or off period
        ooi_next = on_off_indexes[idx[0]+1]
        ts1 = ts_appliance.iloc[ooi]+margin
        ts2 = ts_appliance.iloc[ooi_next]-margin

        # find the indexes into the list of cycle timestamps for the period
        s_indexes = np.where((ts_sigs>=ts1) & (ts_sigs<=ts2))[0]

        if len(s_indexes) > 0:
        # create labelled set, appending labels to the end of the signatures
            labelled_set[c:c+len(s_indexes),0:sig_len] = sigs[s_indexes]
            labelled_set[c:c+len(s_indexes),sig_len] = label
            c = c+len(s_indexes)
            
        label = 1-label ## I think this should be toggled every time, might check
            
    # trucate labelled_set
    labelled_set = labelled_set[:c,:]

    return(labelled_set)


# main function
def get_labelled_flac_file(fn_flac="vi-1451865600_761652.flac", week='/2016/wk01', channel=12):
    dir_flacs = 'D:/flep/ukdale'

    # choose appliance - number 12 is the fridge
    appliance_number = channel

    # display flac date
    ts_start = float(fn_flac[3:13])+float(fn_flac[14:20])/1e6
    date = datetime.utcfromtimestamp(ts_start)
    string = datetime.utcfromtimestamp(ts_start).strftime('%Y-%m-%d %H:%M:%S and %f microseconds')
    print("Starting time of flac file is " + string)

    # get appliance data for the chosen day - 4 Jan 2016 which is the first recording in 2016
    # note that the appliance data is irregularly sampled, with increments of typically 6,7,8 seconds
    number_of_hours = 2
    df_appliance = get_appliance_data(appliance_number, (date.year,date.month,date.day), date.hour)

    # read the flac file (one hour of measurement).  False means that the signal is not converted to volts and amps
    (scaled_voltage,scaled_current) = rff.read_flac(dir_flacs + "/" + fn_flac, False)

    # find the path signatures for each cycle in the voltage signal
    signature_degree = 2
    (zci, sigs) = cycle_sigs(scaled_voltage,scaled_current,signature_degree)
    
    # find the timestamps for the start of each cycle - each sample takes 1/16000 seconds
    ts_sigs = ts_start + zci/16000

    # create set of signatures and labels
    labelled_sigs = create_labelled_set(df_appliance, sigs, ts_sigs)
    return labelled_sigs
    # save a copy for MATLAB
    #fn = '/home/paul/Desktop/cache/dale/code/labelled_sigs.mat'
    #scipy.io.savemat(fn,{'mydata': labelled_sigs})

# function
if __name__ == "__main__":
    #sys.path.append("..")
    #copy = get_labelled_flac_file(fn_flac='vi-1451898000_508684.flac', channel=12)
    print("Done")

Done


More imports and function definitions


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn import ensemble


def get_mains_labelled(fn_flac):
    print("Reading flac file..")
    (voltage,current) = rff.read_flac(dir_flacs + "/" + fn_flac, True)
    
    
    ts_start = float(fn_flac[3:13])+float(fn_flac[14:20])/1e6
    date = datetime.utcfromtimestamp(ts_start)
    string = datetime.utcfromtimestamp(ts_start).strftime('%Y-%m-%d %H:%M:%S and %f microseconds')
    print("Starting time of flac file is " + string)

    fn_dat = "D:/flep/ukdale/house_1/mains.dat"
    start_hour = date.hour+2
    print(start_hour)
    number_of_hours = 1
    ts_start = time.mktime(dt.date(2016,1,4).timetuple()) + 3600*start_hour
    ts_end = ts_start + number_of_hours*3600 
    date = datetime.utcfromtimestamp(ts_start)
    string = datetime.utcfromtimestamp(ts_start).strftime('%Y-%m-%d %H:%M:%S and %f microseconds')
    print("Starting time of csv file is " + string)

    print("Constructing signatures..")
    # find the path signatures for each cycle in the voltage signal
    signature_degree = 2
    (zci, sigs) = cycle_sigs(voltage,current,signature_degree,logsig=True)
    # changing logsig did not have much effect

    # find the timestamps for the start of each cycle - each sample takes 1/16000 seconds
    ts_sigs = ts_start + zci/16000

    # read appliance data
    print("Reading mains.dat in ~30 mb chunks..")
    chunksize = 10**6
    c = 0
    for chunk in pd.read_csv(fn_dat, ' ', chunksize=chunksize):
        if c%10 == 0:
            print(c, '/120')
        chunk.columns = ['ts','watts','a','b']
        c+=1
        aux = chunk.loc[(chunk['ts']>=ts_start) & (chunk['ts']<ts_end),:]
        if aux.size > 0:
            df = aux
    
    print("Shape of df:",df.shape)
    
    #labelling the dataframe with watts
    lst = []
    for idx, ts in np.ndenumerate(df['ts']):
        s_len = sigs.shape[1]
        power = (df.iloc[idx]['watts'])
        
        #get signatures at ts
        s_indexes = np.where((ts_sigs>=ts) & (ts_sigs<=ts+1))[0]
        if len(s_indexes) > 0:
            s = np.zeros(s_len+1)
            s[:s_len] = sigs[s_indexes[0]]
            s[s_len] = power
            lst.append(s)

    filtered_set = np.row_stack(lst)
    return filtered_set

I read the files and created labelled sets for two files, X9 and X14.

In [6]:
# directory name and flac files
dir_flacs = 'D:/flep/ukdale'
print(get_flac_files(dir_flacs))
a = get_flac_files(dir_flacs)[1]
b = get_flac_files(dir_flacs)[2]

data1 = get_mains_labelled(a)
data2 = get_mains_labelled(b)
y9 = data1[:3600,-1]
X9 = data1[:3600,0:-1]
y14 = data2[:3600,-1]
X14 = data2[:3600,0:-1]

['temp', 'vi-1451898000_508684.flac', 'vi-1451916000_275589.flac']
Reading flac file..
Starting time of flac file is 2016-01-04 09:00:00 and 508684 microseconds
11
Starting time of csv file is 2016-01-04 09:00:00 and 000000 microseconds
Constructing signatures..
Reading mains.dat in ~30 mb chunks..
0 /120
10 /120
20 /120
30 /120
40 /120
50 /120
60 /120
70 /120
80 /120
90 /120
100 /120
110 /120
120 /120
Shape of df: (3602, 4)
Reading flac file..
Starting time of flac file is 2016-01-04 14:00:00 and 275589 microseconds
16
Starting time of csv file is 2016-01-04 14:00:00 and 000000 microseconds
Constructing signatures..
Reading mains.dat in ~30 mb chunks..
0 /120
10 /120
20 /120
30 /120
40 /120
50 /120
60 /120
70 /120
80 /120
90 /120
100 /120
110 /120
120 /120
Shape of df: (3603, 4)


Below a few experiments in sklearn. No good results yet:

RandomForestRegressor:
* Overfits a lot. I get ~0.7 scores on the test set if the test set is from the same file as the training set.

LinearRegression:
* Underfits. Scores always below 0.25 I tried scaling but it did not help much.

Increasing signature degree or using logsig does not help. Should I download more files? Also is Havok being updated today?

In [103]:
#X_train, X_test, y_train, y_test = train_test_split(X9, y9, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X24, y14)
#f = LinearRegression()
f = RandomForestRegressor(n_estimators=300, n_jobs=-1, verbose=0)

f.fit(X_train,y_train)
print(f.score(X_train,y_train))
print(f.score(X_test,y_test))
#print(f.score(Z_train,t_train))

8.66474652796434e-05
-0.002483577827505412


In [18]:
y9 = data1[:3600,-1]
X9 = data1[:3600,0:-1]
y14 = data2[:3600,-1]
X14 = data2[:3600,0:-1]
X24 = data2[:3600,1]/320
X29 = data1[:3600,1]/320

X14[:,1]

array([ -5.79811374,  -5.8835712 , -10.83664883, ...,  -7.62075209,
        -7.42156223,  -7.34150669])

In [83]:
(scaled_voltage,scaled_current) = rff.read_flac(dir_flacs + "/" + b, True)
# find zero up-crossing indexes for voltage
zci = np.where(np.diff(np.sign(scaled_voltage))>0)[0]

In [220]:
k = 50*2
def power(second):
    k = 50*second
    return sum(scaled_voltage[zci[k+0]:zci[k+1]]*scaled_current[zci[k+0]:zci[k+1]])/(zci[k+1]-zci[k+0])

In [223]:
powers=[power(s) for s in range(3600)]
df['watts'][:3600]/powers


88154287    1.005329
88154288    1.017048
88154289    0.977775
88154290    0.981028
88154291    0.998779
              ...   
88157882    0.977833
88157883    0.960287
88157884    1.000494
88157885    0.951739
88157886    0.949479
Name: watts, Length: 3600, dtype: float64

In [5]:
fn_dat = "D:/flep/ukdale/house_1/mains.dat"
start_hour = 16
number_of_hours = 1
ts_start = time.mktime(dt.date(2016,1,4).timetuple()) + 3600*start_hour
ts_end = ts_start + number_of_hours*3600 

print("Constructing signatures..")
# find the path signatures for each cycle in the voltage signal
signature_degree = 2
#(zci, sigs) = cycle_sigs(voltage,current,signature_degree,logsig=True)
# changing logsig did not have much effect

# find the timestamps for the start of each cycle - each sample takes 1/16000 seconds
# ts_sigs = ts_start + zci/16000
print(datetime.utcfromtimestamp(ts_start).strftime('%Y-%m-%d %H:%M:%S and %f microseconds'))
# read appliance data
print("Reading mains.dat in ~30 mb chunks..")
chunksize = 10**6
c = 0
for chunk in pd.read_csv(fn_dat, ' ', chunksize=chunksize):
    if c%10 == 0:
        print(c, '/120')
    chunk.columns = ['ts','watts','a','b']
    c+=1
    aux = chunk.loc[(chunk['ts']>=ts_start) & (chunk['ts']<ts_end),:]
    if aux.size > 0:
        df = aux

print("Shape of df:",df.shape)


Constructing signatures..
2016-01-04 14:00:00 and 000000 microseconds
Reading mains.dat in ~30 mb chunks..
0 /120
10 /120
20 /120
30 /120
40 /120
50 /120
60 /120
70 /120
80 /120
90 /120
100 /120
110 /120
120 /120
Shape of df: (3603, 4)


In [222]:
for k in range(0,188000,1000):
    print(sum(scaled_voltage[zci[k+0]:zci[k+1]]*scaled_current[zci[k+0]:zci[k+1]])/(zci[k+1]-zci[k+0]))


530.8609639953478
545.6241001464333
613.3741469240932
633.7084754581014
614.0497884324494
628.876008500472
616.1590149071418
655.5988044734826
625.0618232277784
619.2043028180749
665.3870619565675
582.3515869687177
576.484454447226
766.7703952672357
552.5386371697982
622.1378100101202
665.4955527259002
342.01273624742436
353.1655036936578
354.2190271828373
350.4263039213877
359.7896310570294
355.6661826039325
332.139802390236
332.38223963131776
326.2710940780304
328.911117808904
323.0490260836402
325.8195313369949
322.4282118064984
326.06994162312174
324.68918299298156
322.6365798958884
323.8782994237409
324.5764988244922
323.7655623709947
314.96250122648644
322.7731062525739
346.4976377153024
337.0615113487278
338.55830872015065
352.8370843058834
322.25223550653334
323.40394127381626
322.79708951229486
324.48799698242334
324.1411419115276
323.2849171034803
345.841044043137
326.6564806939793
307.1017035328873
327.1506492976958
324.139596749872
326.68257154105487
327.01390101900535
323.

IndexError: index 182000 is out of bounds for axis 0 with size 181455

In [6]:
truth = [datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S and %f microseconds') for ts in df['ts']]
df['dates'] = truth
df

,ts,watts,a,b,dates
88154287,1.451916e+09,533.69,640.85,245.44,2016-01-04 14:00:00 and 300000 microseconds
88154288,1.451916e+09,535.21,642.77,245.76,2016-01-04 14:00:01 and 300000 microseconds
88154289,1.451916e+09,537.14,644.84,245.81,2016-01-04 14:00:02 and 300000 microseconds
88154290,1.451916e+09,536.57,644.28,245.49,2016-01-04 14:00:03 and 300000 microseconds
88154291,1.451916e+09,534.16,640.60,245.30,2016-01-04 14:00:04 and 300000 microseconds
...,...,...,...,...,...
88157885,1.451920e+09,333.23,433.00,245.78,2016-01-04 14:59:55 and 900000 microseconds
88157886,1.451920e+09,333.16,433.10,245.91,2016-01-04 14:59:56 and 900000 microseconds
88157887,1.451920e+09,332.59,432.50,245.98,2016-01-04 14:59:57 and 900000 microseconds
88157888,1.451920e+09,331.89,432.19,246.10,2016-01-04 14:59:58 and 900000 microseconds


In [8]:
(voltage,current) = rff.read_flac(dir_flacs + "/" + 'vi-1451916000_275589.flac', True)
# find zero up-crossing indexes for voltage
zci = np.where(np.diff(np.sign(voltage))>0)[0]


In [24]:
def cycle_sigs_debug(voltage, current, deg, logsig=True):
    
    # number of channels (dimensions) in path - time, voltage, current
    m = 2

    # find the signature length using an arbitrary path
    sample_path = np.ones([10,m])
    if logsig:
        s = iisignature.prepare(m,deg)
        len_sig = len(iisignature.logsig(sample_path,s))
    else:
        len_sig = len(iisignature.sig(sample_path,deg))

    # find zero up-crossing indexes for voltage
    zci = np.where(np.diff(np.sign(voltage))>0)[0]
    
#debug
    #print('Limiting zci')
    #zci = zci[0:2]

    sigs = np.empty([len(zci)-1,len_sig]);

    # create a path for each cycle 
    for k,z in enumerate(zci[0:-1]):

        ch1 = voltage[zci[k]:zci[k+1]]
        ch2 = current[zci[k]:zci[k+1]] 
        ch2 = np.cumsum(ch1*ch2)
        ch0 = np.linspace(0,1,ch1.shape[0]) # time dimension

        path = np.column_stack([ch0, ch2])

        
        # create the signature using iisignature
        if logsig:
            sigs[k,:] = iisignature.logsig(path,s)
        else:    
            sigs[k,:]  = iisignature.sig(path,deg)
        
            

    # drop the final zci because it is at the cycle end
    zci = zci[0:-1]
    return(zci, sigs)

cycle_sigs_debug(voltage, current, deg=2, logsig=False)

(array([      95,      415,      735, ..., 57558063, 57558382, 57558702],
       dtype=int64),
 array([[1.00000000e+00, 1.69876300e+05, 5.00000000e-01, 8.65055123e+04,
         8.33707878e+04, 1.44289787e+10],
        [1.00000000e+00, 1.74853657e+05, 5.00000000e-01, 8.87367642e+04,
         8.61168927e+04, 1.52869007e+10],
        [1.00000000e+00, 1.74490828e+05, 5.00000000e-01, 8.85611148e+04,
         8.59297136e+04, 1.52235246e+10],
        ...,
        [1.00000000e+00, 1.07340191e+05, 5.00000000e-01, 5.05007718e+04,
         5.68394189e+04, 5.76095827e+09],
        [1.00000000e+00, 1.13196035e+05, 5.00000000e-01, 5.31471898e+04,
         6.00488452e+04, 6.40667116e+09],
        [1.00000000e+00, 1.11897759e+05, 5.00000000e-01, 5.24867520e+04,
         5.94110073e+04, 6.26055427e+09]]))